<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: 1-shot LLAMA-2 Finetuning**

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [4]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [5]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(500))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(500))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 500
Test dataset size: 500


In [6]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [7]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `token_llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `token_

In [9]:
from transformers import AutoTokenizer

max_tok_length = 100

checkpoint = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint, use_auth_token=True,
    padding=True,
    pad_to_multiple_of=8,
    truncation=True,
    max_length=max_tok_length,
    padding_side='left',
    )
tokenizer.pad_token = "[PAD]"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [11]:
train_dataset_filtered = train_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
validation_dataset_filtered = validation_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

Discarding source and target sentences with more than 100 tokens:   0%|          | 0/200 [00:00<?, ? examples/…

Discarding source and target sentences with more than 100 tokens:   0%|          | 0/500 [00:00<?, ? examples/…

Discarding source and target sentences with more than 100 tokens:   0%|          | 0/500 [00:00<?, ? examples/…

In [12]:
print("Training dataset size:", len(train_dataset_filtered))
print("Validation dataset size:", len(validation_dataset_filtered))
print("Test dataset size:", len(test_dataset_filtered))

Training dataset size: 49
Validation dataset size: 116
Test dataset size: 99


In [13]:
tokenized_train_dataset = train_dataset_filtered.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset_filtered.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [14]:
dic = {}
for sample in tokenized_train_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

42   1
47   1
52   2
55   1
56   1
59   1
60   1
61   1
62   1
64   1
65   1
66   2
67   1
68   1
70   4
71   1
72   1
73   1
76   1
79   2
80   2
81   1
82   2
83   2
88   4
89   1
90   1
91   1
93   1
95   2
96   1
98   2
99   2
100   1


In [15]:
import torch

src = "code"
tgt = "docstring"
task_prefix = f"Generate an English {tgt} for the following Python {src}:\n"
s = ""

prefix_tok_len = len(tokenizer.encode(f"{task_prefix}{src}: {s} = {tgt}: "))
max_tok_len = prefix_tok_len
# Adding 2 for new line in target sentence and eos_token_id token
max_tok_len += 2 * max_tok_length + 2


def preprocess4training_function(sample):

    sample_size = len(sample["code"])

    # Creating the prompt with the task description for each source sentence
    inputs  = [f"{task_prefix}{src}: {s} = {tgt}: " for s in sample["code"]]

    # Appending new line after each sample in the batch
    targets = [f"{s}\n" for s in sample["docstring"]]

    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)

    for i in range(sample_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])

    for i in range(sample_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_tok_len - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_tok_len - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_tok_len - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_tok_len])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_tok_len])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_tok_len])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [16]:
sample = tokenized_train_dataset.select(range(2))
model_input = preprocess4training_function(sample)
print(model_input)
print(tokenizer.batch_decode(model_input.input_ids))

{'input_ids': [tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     1,  3251,   403,   385,
         4223,  1574,  1807,   363,   278,  1494,  5132,   775, 29901,    13,
          401, 29901,   822,   903,  8551, 29918, 17281, 29898, 29734, 29918,
        11030,  1125,    13,  1678, 14550,    13,  1678,  9134,   903,  8551,
        29918,  8977,   322,   903, 17281, 29918,

In [17]:
import numpy as np
for i in range(len(model_input['labels'])):
  print(tokenizer.batch_decode([np.where(model_input['labels'][i] < 0, tokenizer.pad_token_id, model_input['labels'][i])]))

['<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

In [18]:
def preprocess4test_function(sample):
    inputs = [f"{task_prefix}{src}: {s} = {tgt}: " for s in sample["code"]]
    model_inputs = tokenizer(inputs,padding=True,)
    return model_inputs

In [19]:
sample = tokenized_train_dataset.select(range(2))
model_input = preprocess4test_function(sample)
print(model_input)
print(tokenizer.batch_decode(model_input.input_ids))

{'input_ids': [[1, 3251, 403, 385, 4223, 1574, 1807, 363, 278, 1494, 5132, 775, 29901, 13, 401, 29901, 822, 903, 8551, 29918, 17281, 29898, 29734, 29918, 11030, 1125, 13, 1678, 14550, 13, 1678, 9134, 903, 8551, 29918, 8977, 322, 903, 17281, 29918, 8149, 297, 263, 2323, 12541, 29889, 13, 1678, 14550, 13, 1678, 736, 9657, 29898, 13, 4706, 313, 7728, 29918, 978, 29892, 3107, 29918, 791, 29897, 13, 4706, 363, 3107, 29918, 978, 29892, 3107, 29918, 791, 297, 4832, 29889, 1524, 7076, 29898, 29734, 29918, 11030, 29897, 13, 4706, 565, 3107, 29918, 978, 451, 297, 903, 3970, 29918, 12256, 29918, 21514, 29909, 1525, 29918, 3738, 6670, 8452, 322, 3107, 29918, 791, 338, 451, 6213, 13, 1678, 1723, 353, 1574, 1807, 29901, 29871], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3251, 403, 385, 4223, 1574, 1807, 363, 278, 1494, 5132, 775, 29901, 13, 401, 29901, 822, 2224, 29898, 9507, 1125, 13, 1678, 9995

In [20]:
preprocessed_train_dataset = tokenized_train_dataset.map(preprocess4training_function, batched=True)
preprocessed_dev_dataset = tokenized_validation_dataset.map(preprocess4training_function, batched=True)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [21]:
for sample in preprocessed_train_dataset.select(range(5)):
    print(sample['input_ids'])
    print(sample['attention_mask'])
    print(sample['labels'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3251, 403, 385, 4223, 1574, 1807, 363, 278, 1494, 5132, 775, 29901, 13, 401, 29901, 822, 903, 8551, 29918, 17281, 29898, 29734, 29918, 11030, 1125, 13, 1678, 14550, 13, 1678, 9134, 903, 8551, 29918, 8977, 322, 903, 17281, 29918, 8149, 297, 263, 2323, 12541, 29889, 13, 1678, 14550, 13, 1678, 736, 9657, 29898, 13, 4706, 313, 7728, 29918, 978, 29892, 3107, 29918, 791, 29897, 13, 4706, 363, 3107, 29918, 978, 29892, 3107, 29918, 791, 297, 4832, 29889, 1524, 7076, 29898, 29734, 29918, 11030, 29897, 13, 4706, 565, 3107, 29918, 978, 451, 297, 903, 3970, 29918, 12256, 29918, 21514, 29909, 1525, 29918, 3738, 6670, 8452, 322, 3107, 29918, 791, 338, 451, 6213, 13, 1678, 1723, 353, 1574, 1807, 29901, 29871, 1, 9134, 903, 8551, 29918, 8977, 3

In [22]:
preprocessed_test_dataset = tokenized_test_dataset.map(preprocess4test_function, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [23]:
for sample in preprocessed_test_dataset.select(range(5)):
    print(sample['input_ids'])
    print(sample['attention_mask'])
    print(sample['labels'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3251, 403, 385, 4223, 1574, 1807, 363, 278, 1494, 5132, 775, 29901, 13, 401, 29901, 822, 2048, 29918, 5552, 29879, 29898, 1311, 29892, 2967, 29918, 5552, 29879, 29892, 4805, 29918, 5552, 29879, 29922, 8516, 1125, 13, 4706, 9995, 8893, 385, 5352, 8600, 1213, 15945, 13, 4706, 12421, 29879, 353, 2967, 29918, 5552, 29879, 29889, 8552, 580, 13, 4706, 565, 4805, 29918, 5552, 29879, 338, 451, 6213, 29901, 13, 9651, 12421, 29879, 29889, 5504, 29898, 17833, 29918, 5552, 29879, 29897, 13, 4706, 736, 12421, 29879, 353, 1574, 1807, 29901, 29871]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 8

In [24]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [25]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [26]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False, gradient_checkpointing_kwargs={'use_reentrant':False})

In [27]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    inference_mode=False,
)

In [28]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


In [29]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8)

### **Training**

In [30]:
from transformers import TrainingArguments

batch_size = 4
gradient_accumulation_steps = 8
model_name = checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir="./",
    run_name="finetuning_sampled_LLAMA",
    eval_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    warmup_steps=100,
    optim="adamw_bnb_8bit",
    prediction_loss_only=True,
    gradient_accumulation_steps = gradient_accumulation_steps,
    bf16=True,
    bf16_full_eval=True,
    group_by_length=True,
)

In [31]:
from transformers import Trainer

trainer = Trainer(
    lora_model,
    args,
    train_dataset=preprocessed_train_dataset,
    eval_dataset=preprocessed_dev_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)


In [32]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
0,No log,2.501747
1,No log,2.500008
2,No log,2.496707


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=3, training_loss=20.91778055826823, metrics={'train_runtime': 846.5969, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.004, 'total_flos': 924720587341824.0, 'train_loss': 20.91778055826823, 'epoch': 2.6153846153846154})

In [33]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}



### **Evaluación**

In [34]:
test_batch_size = 4
batch_tokenized_test = preprocessed_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/99 [00:00<?, ? examples/s]

In [35]:
number_of_batches = len(batch_tokenized_test["input_ids"])
output_sequences = []
for i in range(number_of_batches):
    output_batch = lora_model.generate(
        generation_config=generation_config,
        input_ids=torch.tensor(batch_tokenized_test["input_ids"][i]).cuda(),
        attention_mask=torch.tensor(batch_tokenized_test["attention_mask"][i]).cuda(),
        max_length = max_tok_len,
        num_beams=1,
        do_sample=False,)
    output_sequences.extend(output_batch)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [36]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [37]:
references = tokenizer.batch_decode(preprocessed_test_dataset["labels"], skip_special_tokens=True)

In [38]:
sources = test_dataset_filtered['code']

In [39]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [40]:
from evaluate import load

metric = load("sacrebleu")

In [41]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 12.6


In [42]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [43]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [44]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 54.93 %


In [45]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [46]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 25.11 %
Rouge 2: 23.05 %
Rouge L: 25.14 %
